In [97]:
import torch
import torch.nn as nn

In [137]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        ip = 100
    

        model = [nn.Linear(ip,64),
                     nn.LeakyReLU(0.2),
                      nn.Linear(64,784),
                      nn.ReLU(inplace=True)]
        self.model = torch.nn.Sequential(*model)        
        dmodel = [nn.ConvTranspose2d(4,32, 5, 1,1,bias=False),
                       nn.LeakyReLU(0.2),
                       nn.ConvTranspose2d(32,1, 5, 2,bias=False),
                       nn.Tanh()]
        self.dmodel = torch.nn.Sequential(*dmodel) 

    def forward(self, input):
        
        input = self.model(input)
        input = input.view(64, 4, 14, 14)
        input = self.dmodel(input)
        return input

In [138]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        
        conv = [nn.Conv2d(1,16,5,2), nn.LeakyReLU(0.2),nn.Conv2d(16,128,5,2), nn.LeakyReLU(0.2)]
        self.conv = torch.nn.Sequential(*conv)        
        model = [nn.Linear(4608,64),nn.ReLU(),
                nn.Linear(64,64),nn.Sigmoid(),
                nn.Linear(64,1),nn.Sigmoid()]
        self.model = torch.nn.Sequential(*model) 

    def forward(self, input):
        
        input = self.conv(input)
        input = input.view(-1,4608)
        input = self.model(input)
        return input

In [139]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [140]:
netG = Generator(0)
netG.apply(weights_init)
print(netG)

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=64, out_features=784, bias=True)
    (3): ReLU(inplace)
  )
  (dmodel): Sequential(
    (0): ConvTranspose2d(4, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2)
    (2): ConvTranspose2d(32, 1, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (3): Tanh()
  )
)


In [141]:
netD = Discriminator(0)
netD.apply(weights_init)
print(netD)

Discriminator(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(16, 128, kernel_size=(5, 5), stride=(2, 2))
    (3): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Linear(in_features=4608, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [143]:
from torch.autograd.variable import Variable
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100))
    return n

n = noise(64)
print(n.shape)
fake = netG(n)
print(fake.shape)
output = netD(fake)
print(output.shape)

torch.Size([64, 100])
torch.Size([64, 1, 35, 35])
torch.Size([64, 1])


In [32]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        
        
       
        model=[]
        model += [nn.ConvTranspose2d(4,64, 3, 1,bias=False),nn.ReLU(inplace=True)]
        model += [nn.ConvTranspose2d(64,1, 3, 2,bias=False),nn.Tanh()]
        self.main = torch.nn.Sequential(*model)

    def forward(self, input):
        return self.main(input)

netg = Generator(1)
noise = torch.randn(64, 4, 1, 1)

fake = netg(noise)